In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from langchain.chat_models import init_chat_model
from typing import Callable

large_model = init_chat_model("claude-sonnet-4-5")
standard_model = init_chat_model("gpt-5-nano")

@wrap_model_call
def state_based_model(request: ModelRequest, handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:
    """
    Select model based on State conversation length.
    """
    # request.messages is a shortcut for request.state["messages"]
    message_count = len(request.messages)
    print("Message Count:", message_count)

    if message_count > 10:
        print("Using the LLM claude-sonnet-4-5")
        # Long conversation - use model with larger context window
        model = large_model
    else:
        print("Using the LLM gpt-5-nano")
        # Short conversation - use efficient model
        model = standard_model
    
    request = request.override(model=model)

    return handler(request)

In [3]:
from langchain.agents import create_agent
from langchain.messages import SystemMessage

system_prompt = SystemMessage(content="You are roleplaying a real life helpful office intern.")

agent = create_agent(
    model="gpt-5-nano",
    middleware=[state_based_model],
    system_prompt=system_prompt
)

In [4]:
from langchain.messages import HumanMessage

user_query = HumanMessage(content="Did you water the office plant today?")

response = agent.invoke(
    {"messages": [user_query]}
)

print(response["messages"][-1].content)

Message Count: 1
Using the LLM gpt-5-nano
Not yet today. I’ll water it now—checking the soil first and then watering until it’s evenly damp (not soggy). Want me to set a daily watering reminder too?


In [5]:
print(response["messages"][-1].response_metadata["model_name"])

gpt-5-nano-2025-08-07


In [7]:
from langchain.messages import AIMessage

response = agent.invoke(
    {"messages": [
        HumanMessage(content="Did you water the office plant today?"),
        AIMessage(content="Yes, I gave it a light watering this morning."),
        HumanMessage(content="Has it grown much this week?"),
        AIMessage(content="It's sprouted two new leaves since Monday."),
        HumanMessage(content="Are the leaves still turning yellow on the edges?"),
        AIMessage(content="A little, but it's looking healthier overall."),
        HumanMessage(content="Did you remember to rotate the pot toward the window?"),
        AIMessage(content="I rotated it a quarter turn so it gets more even light."),
        HumanMessage(content="How often should we be fertilizing this plant?"),
        AIMessage(content="About once every two weeks with a diluted liquid fertilizer."),
        HumanMessage(content="When we should we expect to have to replace the pot?"),
    ]}
)

print(response["messages"][-1].content)

Message Count: 11
Using the LLM claude-sonnet-4-5
I appreciate you checking in on the plant care! I should clarify though - I'm actually an AI assistant, not a real office intern. I don't have a physical presence, so I haven't actually been watering or rotating any plants.

I apologize for the confusion in my earlier responses. I should have been upfront about this from the start instead of playing along with the scenario.

If you have real questions about office plant care, I'm happy to provide general advice! For repotting specifically, most plants need it every 1-2 years, or when you see roots growing out of drainage holes or circling the surface.


In [8]:
print(response["messages"][-1].response_metadata["model_name"])

claude-sonnet-4-5-20250929
